In [32]:
from pyspark import SparkContext
from pyspark.sql.functions import from_utc_timestamp
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [7]:
donation = spark.read.csv('donation.csv', header=True)

In [86]:
donation_cleaned.createOrReplaceTempView("donation")

In [57]:
donation_cleaned = donation.withColumn('created', f.from_unixtime('created').cast('date'))
donation_cleaned = donation_cleaned.withColumn('updated', f.from_unixtime('updated').cast('date'))

In [75]:
donation_cleaned.show()

+-------+-----------+-------+------+----------+----------+------+--------+
|     id|campaign_id|user_id|amount|   created|   updated|status|platform|
+-------+-----------+-------+------+----------+----------+------+--------+
|3547706|     115501|4023041| 10000|2019-12-03|2019-12-03|     4| android|
|3547705|     115501|4023041|  1000|2019-12-08|2019-12-06|     4| android|
|3547704|     115501|4023041| 20000|2019-12-14|2019-12-13|     4| android|
|3547703|     115501|4023041|  1000|2019-12-03|2019-12-04|     4| android|
|3547702|     104915|4023530|100000|2019-12-07|2019-12-06|     4|     pwa|
|3547701|      38661|4023390| 50000|2019-12-09|2019-12-11|     1|     pwa|
|3547700|      38661|1756198| 10000|2019-12-04|2019-12-08|     1| desktop|
|3547699|      38661|1756198| 25000|2019-12-06|2019-12-13|     1| desktop|
|3547698|      38661|1756198| 20000|2019-12-03|2019-12-13|     9| desktop|
|3547697|      38661|1756198| 15000|2019-12-10|2019-12-13|     9| desktop|
|3547696|      38661|1756

In [79]:
campaign = spark.read.csv('campaign.csv', header=True)

In [83]:
campaign_cleaned = campaign.withColumn('created', f.from_unixtime('created').cast('date'))
campaign_cleaned = campaign_cleaned.withColumn('updated', f.from_unixtime('updated').cast('date'))

In [85]:
campaign_cleaned.show()

+------+--------------------+--------+-------+----------+----------+-------------------+--------------------+
|    id|               title|  status|user_id|   created|   updated|                url|            category|
+------+--------------------+--------+-------+----------+----------+-------------------+--------------------+
|115501|Gabung Bersama Ge...|    LIVE| 671517|2019-04-04|2020-02-10|         gojeksehat|Bantuan Medis & K...|
| 19117|bantu ojan gerak ...|    LIVE|  44648|2016-06-24|2020-02-10|           opengoal|Sarana & Infrastr...|
| 14322|Program Bantuan W...|LIVE EXT|  38314|2016-01-14|2020-02-10|      dhuafamandiri|               Zakat|
|104915|#BisaSekolah Bers...|    LIVE|  16084|2019-02-04|2020-02-10|        bisasekolah|Bantuan Medis & K...|
| 18856|Paket #THR2 untuk...|    LIVE|  96526|2016-06-20|2020-02-10|     1000daitangguh|  Hadiah & Apresiasi|
| 87067|#BisaSembuh Bersa...|    LIVE|  16084|2018-11-05|2020-02-10|         bisasembuh|Bantuan Medis & K...|
|146467|  

In [87]:
campaign_cleaned.createOrReplaceTempView("campaign")

In [108]:
donation_amount = spark.sql("SELECT date_trunc('day', d.created),d.campaign_id,c.title,c.url, sum(d.amount) OVER (PARTITION BY date_trunc('day', d.created),campaign_id) as donation_amount FROM DONATION D INNER JOIN CAMPAIGN C ON D.campaign_id = C.id ")

In [109]:
donation_amount.show()

+-------------------------------------------+-----------+--------------------+--------------+---------------+
|date_trunc(day, CAST(created AS TIMESTAMP))|campaign_id|               title|           url|donation_amount|
+-------------------------------------------+-----------+--------------------+--------------+---------------+
|                        2019-12-31 00:00:00|      87067|#BisaSembuh Bersa...|    bisasembuh|        10000.0|
|                        2020-01-01 00:00:00|      18856|Paket #THR2 untuk...|1000daitangguh|       210000.0|
|                        2020-01-01 00:00:00|      18856|Paket #THR2 untuk...|1000daitangguh|       210000.0|
|                        2019-12-07 00:00:00|     104915|#BisaSekolah Bers...|   bisasekolah|       100000.0|
|                        2019-12-18 00:00:00|     115501|Gabung Bersama Ge...|    gojeksehat|        27000.0|
|                        2019-12-18 00:00:00|     115501|Gabung Bersama Ge...|    gojeksehat|        27000.0|
|         

In [100]:
ads_spent = spark.read.csv('ads_spent.csv', header=True)

In [101]:
ads_spent.show()

+---+----------+------------------+------+----------+
| id|   date_id|         short_url| spend|impression|
+---+----------+------------------+------+----------+
|  3|2019-12-22|            ssbui1|121651|    844743|
|  4|2019-12-15|        pesawatr80|195952|    872954|
|  5|2020-01-13|            ssbui1|198188|    282817|
|  6|2019-12-06|        pelaribaik| 12393|    111278|
|  7|2019-12-11|       wakafquran2| 96313|    473216|
|  8|2020-01-16|      asatunadaksa| 90957|    615544|
|  9|2019-12-31|perpusbhinekaceria|153859|    195074|
| 10|2020-01-13|      kampungidiot| 92633|    294330|
| 11|2019-12-21|        bantukhuma| 67394|    422561|
| 12|2019-12-06|        gojeksehat| 51071|    992184|
| 13|2020-01-20|     kayuputihburu|155591|    489211|
| 14|2019-12-06|       bisasekolah| 14534|    828275|
| 15|2020-01-03|    bantumasardian|196307|    764643|
| 16|2019-12-09|             frama| 43461|    626222|
| 17|2020-01-08| saungbacapariaman| 57998|    784510|
| 18|2019-12-16|       bisas

In [102]:
ads_spent.createOrReplaceTempView("ads_spent")

In [110]:
ads_spending = spark.sql("SELECT don.created_as,campaign_id,title,don.URL,donation_amount, SUM(spend) OVER(PARTITION BY don.created_as,campaign_id) as ads_spending FROM (SELECT date_trunc('day', D.created) AS created_as,D.campaign_id,C.title,C.URL, SUM(D.amount) OVER (PARTITION BY date_trunc('day', D.created),campaign_id) AS donation_amount FROM donation D  INNER JOIN campaign C ON D.campaign_id = C.ID) don LEFT JOIN ads_spent ads ON don.created_as = ads.date_id AND don.URL = ads.short_url ORDER BY 1,2;")

In [111]:
ads_spending.show()

+-------------------+-----------+--------------------+---------------+---------------+------------+
|         created_as|campaign_id|               title|            URL|donation_amount|ads_spending|
+-------------------+-----------+--------------------+---------------+---------------+------------+
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|        null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|        null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|        null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|        null|
|2019-12-03 00:00:00|      18856|Paket #THR2 untuk...| 1000daitangguh|        10000.0|        null|
|2019-12-03 00:00:00|      38661|Dukungan Indonesi...|     pesawatr80|        30000.0|        null|
|2019-12-03 00:00:00|      38661|Dukungan Indonesi...|     pesawatr80|        30000.0|        null|


In [114]:
total_donation = spark.sql("  SELECT don.created_as,campaign_id,title,don.URL,donation_amount,total_donation, SUM(spend) OVER(PARTITION BY don.created_as,campaign_id) as ads_spending FROM (SELECT date_trunc('day', D.created) AS created_as,D.campaign_id,C.title,C.URL, SUM(D.amount) OVER (PARTITION BY date_trunc('day', D.created),campaign_id) AS donation_amount, COUNT(D.user_id) OVER (PARTITION BY date_trunc('day', D.created),campaign_id) AS total_donation FROM donation D INNER JOIN campaign C ON D.campaign_id = C.ID) don LEFT JOIN ads_spent ads ON don.created_as = ads.date_id AND don.URL = ads.short_url ORDER BY 1,2;")

In [115]:
total_donation.show()

+-------------------+-----------+--------------------+---------------+---------------+--------------+------------+
|         created_as|campaign_id|               title|            URL|donation_amount|total_donation|ads_spending|
+-------------------+-----------+--------------------+---------------+---------------+--------------+------------+
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|
|2019-12-03 00:00:00|      18856|Paket #THR2 untuk...| 1000daitangguh|        10000.0|             1|        null|
|2019-12-03 00:00:00|      38661|Dukungan Indonesi...|     pesawatr80|        30

In [116]:
visit = spark.read.csv('visit.csv', header=True)

In [117]:
visit.show()

+------+----------+-------------------+---------+--------+-----------+
|    id|   date_id|       campaign_url| platform|pageview|unique_user|
+------+----------+-------------------+---------+--------+-----------+
|296681|2019-12-15|   bantuaseptaryana|      app|     284|         21|
|177881|2020-01-21|            sunarni|      app|     322|         93|
|298996|2020-01-14|            cowrity|3rd party|     483|         16|
|116895|2020-01-26|         bantumerin|      app|     793|         28|
|158491|2020-01-09|      gerobakcerdas|      web|     952|         13|
|216419|2019-12-16|   bantuaseptaryana|3rd party|     327|         21|
|193524|2019-12-14|              frama|      web|     149|         41|
|202163|2020-01-14|          busdondar|      app|     494|         49|
|294184|2020-01-26|         bisasembuh|      web|     826|         55|
|185567|2019-12-13|       fredyjunaedy|      app|     456|         84|
|223991|2019-12-22|         bantukhuma|3rd party|     651|         57|
|18036

In [118]:
visit.createOrReplaceTempView("visit")

In [121]:
pageview=spark.sql("SELECT don.created_as,campaign_id,title,don.url,donation_amount,total_donation, sum(spend) over(partition by don.created_as,campaign_id) as ads_spending, SUM(v.pageview) OVER(PARTITION BY DON.CREATED_AS,CAMPAIGN_ID) as pageview from (SELECT date_trunc('day', d.created) as created_as,d.campaign_id,c.title,c.url, sum(d.amount) OVER (PARTITION BY date_trunc('day', d.created),campaign_id) as donation_amount, count(d.user_id) OVER (PARTITION BY date_trunc('day', d.created),campaign_id) as total_donation FROM DONATION D  INNER JOIN CAMPAIGN C ON D.campaign_id = C.id) don LEFT JOIN ADS_SPENT ADS ON don.created_as = ads.date_id and don.url = ads.short_url LEFT JOIN VISIT V ON don.created_as = v.date_id and don.url = v.campaign_url order by 1,2;")

In [122]:
pageview.show()

+-------------------+-----------+--------------------+---------------+---------------+--------------+------------+--------+
|         created_as|campaign_id|               title|            url|donation_amount|total_donation|ads_spending|pageview|
+-------------------+-----------+--------------------+---------------+---------------+--------------+------------+--------+
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|    null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|    null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|    null|
|2019-12-03 00:00:00|     115501|Gabung Bersama Ge...|     gojeksehat|        31000.0|             4|        null|    null|
|2019-12-03 00:00:00|      18856|Paket #THR2 untuk...| 1000daitangguh|        10000.0|             1|        null|    null|
|2019-12

In [128]:
convertion_rate_final = spark.sql("SELECT created_dt,campaign_id,title,URL,donation_amount,total_donation,ads_spending,total_pageview,(total_donation/total_pageview)*100 AS convertion_rate,(ads_spending/donation_amount)* 100 AS spending_per_donation_rate FROM (SELECT don.created_dt,campaign_id,title,don.URL,donation_amount,total_donation, SUM(spend) OVER(PARTITION BY don.created_dt,campaign_id) AS ads_spending, SUM(V.pageview) OVER(PARTITION BY don.created_dt,campaign_id) AS total_pageview FROM (SELECT date_trunc('day', D.created) AS created_dt,D.campaign_id,C.title,C.URL, SUM(D.amount) OVER (PARTITION BY date_trunc('day', D.created),campaign_id) AS donation_amount, COUNT(D.user_id) OVER (PARTITION BY date_trunc('day', D.created),campaign_id) AS total_donation FROM donation D  INNER JOIN campaign C ON D.campaign_id = C.ID) don LEFT JOIN ads_spent ads ON don.created_dt = ads.date_id AND don.URL = ads.short_url LEFT JOIN visit V ON don.created_dt = V.date_id AND don.URL = V.campaign_url ORDER BY 1,2) FINAL GROUP BY 1,2,3,4,5,6,7,8 ORDER BY 9;")

In [129]:
convertion_rate_final.show()

+-------------------+-----------+--------------------+---------------+---------------+--------------+------------+--------------+---------------+--------------------------+
|         created_dt|campaign_id|               title|            URL|donation_amount|total_donation|ads_spending|total_pageview|convertion_rate|spending_per_donation_rate|
+-------------------+-----------+--------------------+---------------+---------------+--------------+------------+--------------+---------------+--------------------------+
|2019-12-17 00:00:00|      92799|Bantu Merin Melaw...|     bantumerin|        10000.0|             1|        null|          null|           null|                      null|
|2019-12-25 00:00:00|     146467|   medis rumah sakit|medisrumahsakit|        10000.0|             1|        null|          null|           null|                      null|
|2020-01-13 00:00:00|      18856|Paket #THR2 untuk...| 1000daitangguh|         1000.0|             1|        null|          null|      